In [25]:
from xgboost import XGBClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from xgboost import plot_tree

In [26]:
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size=.2,  random_state=101)

xgbc = XGBClassifier(n_estimators=1, max_depth=1, learning_rate=1, objective='binary:logistic')
xgbc.fit(X_train, y_train)

preds = xgbc.predict(X_test)
preds

array([0, 0, 0, 1, 1, 2, 1, 1, 2, 0, 2, 0, 0, 2, 2, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 2, 0, 0])

In [27]:
booster = xgbc.get_booster()
booster

In [28]:
# for i in range(3):
#        plot_tree(xgbc, num_trees=i)

In [29]:
trees = booster.get_dump(with_stats=True)
for i, tree in enumerate(trees):
  print(f"Tree {i}:")
  print(tree)

Tree 0:
0:[f2<3] yes=1,no=2,missing=2,gain=57.3227081,cover=53.3333282
	1:leaf=1.42011833,cover=17.7777767
	2:leaf=-0.729483366,cover=35.5555534

Tree 1:
0:[f2<3] yes=1,no=2,missing=2,gain=12.9075136,cover=53.3333282
	1:leaf=-0.710059226,cover=17.7777767
	2:leaf=0.310030371,cover=35.5555534

Tree 2:
0:[f3<1.70000005] yes=1,no=2,missing=2,gain=50.4487953,cover=53.3333282
	1:leaf=-0.672897279,cover=34.6666641
	2:leaf=1.32203388,cover=18.666666



In [30]:
booster.get_dump()

['0:[f2<3] yes=1,no=2,missing=2\n\t1:leaf=1.42011833\n\t2:leaf=-0.729483366\n',
 '0:[f2<3] yes=1,no=2,missing=2\n\t1:leaf=-0.710059226\n\t2:leaf=0.310030371\n',
 '0:[f3<1.70000005] yes=1,no=2,missing=2\n\t1:leaf=-0.672897279\n\t2:leaf=1.32203388\n']

In [31]:
print(type(booster))

<class 'xgboost.core.Booster'>


In [32]:
X_test[0]

array([5.5, 4.2, 1.4, 0.2])

In [33]:
sum = 0
for i in range(3):
  print(xgbc.predict_proba(X_test[[0]])[0][i])
  sum += xgbc.predict_proba(X_test[[0]])[0][i]
print(sum)

0.80506814
0.09565514
0.09927674
1.0


In [34]:
import numpy as np

vector = np.array([-0.7302, 0.338, -0.699])

min_value = np.min(vector)
shifted_vector = vector - min_value 
normalized_vector = shifted_vector / np.sum(shifted_vector)

print(normalized_vector)
print(np.sum(normalized_vector))

[0.         0.97162088 0.02837912]
0.9999999999999999
